## Exploration of base dataset - Adult

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
DATA_DIR = Path("/opt/anaconda3/lib/python3.11/site-packages/aif360/data/raw/adult")

# 2. Column names (from adult.names)
COLUMNS = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

# 3. Read train & test, then concatenate
train = pd.read_csv(DATA_DIR/"adult.data", names=COLUMNS,
                    sep=r',\s*', engine='python', na_values='?')
test  = pd.read_csv(DATA_DIR/"adult.test", names=COLUMNS,
                    sep=r',\s*', engine='python', skiprows=1, na_values='?')

df = pd.concat([train, test], ignore_index=True)

In [3]:
pd.set_option('display.max_colwidth', None)
top_n = 5
rows = []
for col in df.columns:
    feature_label = 'Target: income' if col == 'income' else col
    ser = df[col]

    if pd.api.types.is_numeric_dtype(ser):
        rows.append({
            'Feature': feature_label,
            'Type': 'Continuous',
            'Min': ser.min(),
            'Max': ser.max(),
            'Mean': round(ser.mean(), 2),
            'Std': round(ser.std(), 2),
            'Values': ''
        })
    else:
        counts = ser.dropna().value_counts()
        if counts.size > top_n:
            vals = counts.index[:top_n].tolist() + ['Other']
        else:
            vals = counts.index.tolist()
        rows.append({
            'Feature': feature_label,
            'Type': 'Categorical',
            'Min': '',
            'Max': '',
            'Mean': '',
            'Std': '',
            'Values': ", ".join(map(str, vals))
        })

summary_df = pd.DataFrame(rows)[['Feature', 'Type', 'Min', 'Max', 'Mean', 'Std', 'Values']]

# Reset index to start at 1
summary_df.index = range(1, len(summary_df) + 1)
summary_df

,Feature,Type,Min,Max,Mean,Std,Values
1,age,Continuous,17,90,38.64,13.71,
2,workclass,Categorical,,,,,"Private, Self-emp-not-inc, Local-gov, State-gov, Self-emp-inc, Other"
3,fnlwgt,Continuous,12285,1490400,189664.13,105604.03,
4,education,Categorical,,,,,"HS-grad, Some-college, Bachelors, Masters, Assoc-voc, Other"
5,education-num,Continuous,1,16,10.08,2.57,
6,marital-status,Categorical,,,,,"Married-civ-spouse, Never-married, Divorced, Separated, Widowed, Other"
7,occupation,Categorical,,,,,"Prof-specialty, Craft-repair, Exec-managerial, Adm-clerical, Sales, Other"
8,relationship,Categorical,,,,,"Husband, Not-in-family, Own-child, Unmarried, Wife, Other"
9,race,Categorical,,,,,"White, Black, Asian-Pac-Islander, Amer-Indian-Eskimo, Other"
10,sex,Categorical,,,,,"Male, Female"


In [4]:
df['income'] = df['income'].str.strip().str.rstrip('.')

n_instances = len(df)
n_features  = df.shape[1] - 1  
positive_rate = df['income'].eq('>50K').mean() * 100

# Positive-class rate by sex
sex_rates = (
    df
    .groupby('sex')['income']
    .apply(lambda s: s.eq('>50K').mean() * 100)
    .round(1)
)

# Positive-class rate by race
race_rates = (
    df
    .groupby('race')['income']
    .apply(lambda s: s.eq('>50K').mean() * 100)
    .round(1)
)

# Define White vs. Non-White
df['race_group'] = np.where(df['race'] == 'White', 'White', 'Non-White')

# Positive-class rate by race_group
racegrp_rates = (
    df
    .groupby('race_group')['income']
    .apply(lambda s: s.eq('>50K').mean() * 100)
    .round(1)
)

# Intersectional 2×2 matrix (race_group × sex) out of curiosity
matrix = (
    df
    .assign(positive=df['income'].eq('>50K').astype(int))
    .pivot_table(
        index='race_group',
        columns='sex',
        values='positive',
        aggfunc=lambda x: x.mean() * 100
    )
    .round(1)
)

# Print all summaries
print(f"Instances:        {n_instances}")
print(f"Features:         {n_features}")
print(f"Positive-class:   {positive_rate:.1f}%")
print(f"Protected attrs:  race, sex")

print("\n Positive-class rate by race")
print(race_rates.astype(str) + '%')

print("\n Positive-class rate by sex")
print(sex_rates.astype(str) + '%')

print("\nPositive-class rate (White vs. Non-White)")
print(racegrp_rates.astype(str) + '%')

print("\nIntersection (Race × Sex)")
print(matrix.astype(str) + '%')

Instances:        48842
Features:         14
Positive-class:   23.9%
Protected attrs:  race, sex

 Positive-class rate by race
race
Amer-Indian-Eskimo    11.7%
Asian-Pac-Islander    26.9%
Black                 12.1%
Other                 12.3%
White                 25.4%
Name: income, dtype: object

 Positive-class rate by sex
sex
Female    10.9%
Male      30.4%
Name: income, dtype: object

Positive-class rate (White vs. Non-White)
race_group
Non-White    15.3%
White        25.4%
Name: income, dtype: object

Intersection (Race × Sex)
sex        Female   Male
race_group              
Non-White    7.2%  21.8%
White       11.8%  31.5%


In [5]:
race_counts = df['race'].value_counts()
race_perc   = df['race'].value_counts(normalize=True).mul(100).round(1)

race_summary = pd.DataFrame({
    'Count': race_counts,
    'Percentage (%)': race_perc
})

race_summary

,Count,Percentage (%)
race,,
White,41762,85.5
Black,4685,9.6
Asian-Pac-Islander,1519,3.1
Amer-Indian-Eskimo,470,1.0
Other,406,0.8
